<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220524.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* regressor

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotnine as gg
from plotnine import ggplot, aes
from datetime import datetime, time, date, timezone, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA 
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, mean_squared_error
import sklearn.datasets as skd

# regressor

In [3]:
raw = pd.read_csv('train.csv')
raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
raw.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
raw.columns = [a.lower() for a in raw.columns]
raw.head(1)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [7]:
raw = raw.drop(['passengerid', 'cabin'], axis = 1)
raw.head(1)

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,S


In [63]:
t1 = raw['ticket'].str.extract('(.+)[^\d{1,}]')
t1.columns = ['ticket']
t1 = t1.assign(ticket = lambda x : np.where(x['ticket'].isnull(), 'X', x['ticket'].str.lower()))
t2 = t1
t2 = t2.assign(t2 = t2['ticket'].str.replace('\.|\/','', regex = True) )['t2'].value_counts().reset_index()
t2 = t2.assign(index2 = np.where(t2['t2'] >= 6, t2['index'], 'etc'))
t2 = t2.drop('t2',axis = 1)
t2.columns = ['ticket2', 'ticket3']
t2.head()

,ticket2,ticket3
0,X,X
1,pc,pc
2,ca,ca
3,a5,a5
4,sotonoq,sotonoq


In [71]:
getter = dict(zip(t2['ticket2'],t2['ticket3']))
getter

{'X': 'X',
 'a4': 'a4',
 'a5': 'a5',
 'as': 'etc',
 'c': 'etc',
 'ca': 'ca',
 'casoton': 'etc',
 'fa': 'etc',
 'fc': 'etc',
 'fcc': 'etc',
 'lin': 'etc',
 'pc': 'pc',
 'pp': 'etc',
 'ppp': 'etc',
 'sc': 'etc',
 'sca4': 'etc',
 'scah': 'etc',
 'scah basle': 'etc',
 'scow': 'etc',
 'scparis': 'scparis',
 'soc': 'soc',
 'sop': 'etc',
 'sopp': 'etc',
 'sotono2': 'etc',
 'sotonoq': 'sotonoq',
 'sp': 'etc',
 'stono 2': 'stono 2',
 'stono2': 'stono2',
 'swpp': 'etc',
 'wc': 'wc',
 'wep': 'etc'}

In [95]:
raw2 = raw.assign(ticket2 = raw['ticket'].str.extract('(.+)[^\d{1,}]'),
           ticket_num = raw['ticket'].str.extract('(^\d{1,})')
           )
raw2.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,ticket2,ticket_num
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,A/5,NaN
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,PC,NaN
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,STON/O2.,NaN
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,NaN,113803
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,373450


In [96]:
raw2 = raw2.assign(ticket2 = lambda x : np.where(x['ticket2'].isnull(), 'X', x['ticket2'].str.lower().str.replace('\.|\/|\s', '')))
raw2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,ticket2,ticket_num
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,a5,NaN
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,pc,NaN
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,stono2,NaN
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,X,113803
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,X,373450


In [97]:
raw2 = raw2.applymap(lambda x : getter.get(x, x))
raw2 = raw2.assign(ticket_num = lambda x : np.where(x['ticket_num'].isnull(), 0, x['ticket_num']))
raw2 = raw2.drop(['ticket', 'name'], axis = 1)
raw2 = raw2.assign(fam = raw2.sibsp + raw2.parch + 1)
raw2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,ticket2,ticket_num,fam
0,0,3,male,22.0,1,0,7.2500,S,a5,0,2
1,1,1,female,38.0,1,0,71.2833,C,pc,0,2
2,1,3,female,26.0,0,0,7.9250,S,stono2,0,1
3,1,1,female,35.0,1,0,53.1000,S,X,113803,2
4,0,3,male,35.0,0,0,8.0500,S,X,373450,1


In [105]:
raw3 = pd.get_dummies(raw2, columns = ['pclass','sex','embarked','ticket2'])
raw3.columns

Index(['survived', 'age', 'sibsp', 'parch', 'fare', 'ticket_num', 'fam',
       'pclass_1', 'pclass_2', 'pclass_3', 'sex_female', 'sex_male',
       'embarked_C', 'embarked_Q', 'embarked_S', 'ticket2_X', 'ticket2_a4',
       'ticket2_a5', 'ticket2_ca', 'ticket2_etc', 'ticket2_pc',
       'ticket2_scahbasle', 'ticket2_scparis', 'ticket2_soc',
       'ticket2_sotonoq', 'ticket2_stono2', 'ticket2_wc'],
      dtype='object')

In [109]:
imputer = KNNImputer(n_neighbors = 3)
raw4 = pd.DataFrame(imputer.fit_transform(raw3), columns = raw3.columns)
raw4.head()

,survived,age,sibsp,parch,fare,ticket_num,fam,pclass_1,pclass_2,pclass_3,...,ticket2_a5,ticket2_ca,ticket2_etc,ticket2_pc,ticket2_scahbasle,ticket2_scparis,ticket2_soc,ticket2_sotonoq,ticket2_stono2,ticket2_wc
0,0.0,22.0,1.0,0.0,7.2500,0.0,2.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,26.0,0.0,0.0,7.9250,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,113803.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,35.0,0.0,0.0,8.0500,373450.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
raw4.isnull().sum()

survived             0
age                  0
sibsp                0
parch                0
fare                 0
ticket_num           0
fam                  0
pclass_1             0
pclass_2             0
pclass_3             0
sex_female           0
sex_male             0
embarked_C           0
embarked_Q           0
embarked_S           0
ticket2_X            0
ticket2_a4           0
ticket2_a5           0
ticket2_ca           0
ticket2_etc          0
ticket2_pc           0
ticket2_scahbasle    0
ticket2_scparis      0
ticket2_soc          0
ticket2_sotonoq      0
ticket2_stono2       0
ticket2_wc           0
dtype: int64

In [113]:
X = raw4[[ 'age', 'sibsp', 'parch', 'fare', 'ticket_num', 'fam',
       'pclass_1', 'pclass_2', 'pclass_3', 'sex_female', 'sex_male',
       'embarked_C', 'embarked_Q', 'embarked_S', 'ticket2_X', 'ticket2_a4',
       'ticket2_a5', 'ticket2_ca', 'ticket2_etc', 'ticket2_pc',
       'ticket2_scahbasle', 'ticket2_scparis', 'ticket2_soc',
       'ticket2_sotonoq', 'ticket2_stono2', 'ticket2_wc']]
y = raw4['survived']

In [114]:
x_tr, x_te, y_tr, y_te = train_test_split(X,y, train_size = .7)

In [115]:
fit_rf = RandomForestClassifier().fit(x_tr, y_tr)

In [118]:
accuracy_score(y_tr, fit_rf.predict(x_tr)), accuracy_score(y_te, fit_rf.predict(x_te))

(0.9983948635634029, 0.8097014925373134)